Make sure you you download the archive from https://www.kaggle.com/datasets/balraj98/deepglobe-road-extraction-dataset/code
and rename the folder to DeepGlobe_Road_Extraction. This script samples images of that dataset, divides the image in 4 images, resizes the images to (400,400) and only keep the images that have a segmentation mask of over 7.5% roads and saves it into png format. Make sure
you have the target folder DeepGlobeSampled and inside images and grountruth folder where the images are copied into.

In [10]:
!pip install Pillow


[notice] A new release of pip available: 22.3 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import random
from PIL import Image
os.getcwd()

entries = os.listdir('DeepGlobe_Road_Extraction/train')
entries = [string for string in entries if string.endswith("_sat.jpg")]

print(len(entries))

#sampled_entries = ['934063_sat.jpg', '719559_sat.jpg', '941690_sat.jpg', '163227_sat.jpg', '234752_sat.jpg', '124319_sat.jpg', '938177_sat.jpg', '660195_sat.jpg', '159264_sat.jpg', '356160_sat.jpg']
#sampled_entries = entries
print(sampled_entries)
print(len(sampled_entries))

6226
['934063_sat.jpg', '719559_sat.jpg', '941690_sat.jpg', '163227_sat.jpg', '234752_sat.jpg', '124319_sat.jpg', '938177_sat.jpg', '660195_sat.jpg', '159264_sat.jpg', '356160_sat.jpg']
10


In [2]:
def split_in_quarters(image):
    im1 = image.crop((0, 0, 512, 512)) # top left corner
    im2 = image.crop((512, 0, 1024, 512)) # top right corner
    im3 = image.crop((0, 512, 512, 1024)) # bottom left corner
    im4 = image.crop((512, 512, 1024, 1024)) # bottom left corner

    # put them into array

    images = [im1, im2, im3, im4]
    # resize them all to 400x400
    for i in range(0, len(images)):
        images[i] = images[i].resize((400,400))

    return images

In [3]:
import numpy as np
folder = 'DeepGlobe_Road_Extraction/train/'
img_name = sampled_entries[0]
print(img_name)
id = img_name.split("_")[0] # orignal shape (1024, 1024)

mask_name = id + '_mask.png'

img_path = folder + img_name
img_path = os.path.join(os.getcwd(), img_path)

mask_path = folder  + mask_name
mask_path = os.path.join(os.getcwd(), mask_path)


image = Image.open(img_path)
#resized_image = image.resize((400, 400))

images = split_in_quarters(image)

mask = Image.open(mask_path)
#resized_mask = mask.resize((400, 400))
grey_mask = mask.convert('L')
masks = split_in_quarters(grey_mask)

np_img = np.asarray(grey_mask)
print(np_img.shape)

new_name = 'satimage_' + id + '.png'

#resized_image.save('DeepGlobeSampled/images/' + new_name)
#resized_mask.save('DeepGlobeSampled/groundtruth/' + new_name)
print(images[0].size)
masks[0].getpixel((1,1))

934063_sat.jpg
(1024, 1024)
(400, 400)


0

In [3]:
def check_if_contains_road(mask):

    width, height = mask.size
    ctr = 0
    threshold = 12000 # only keep ones with over average of our dataset, average is 28611
    for y in range(height):
        for x in range(width):
            # Check if the pixel value is 255 (white)
            if mask.getpixel((x, y)) == 255:
                ctr += 1

    if ctr >= threshold: # only return true if we are over threshold
        return True
    else:
        return False
    

In [4]:
def binarize(masks):
    binarized = []
    threshold = 128 # put all pixels that are larger than 128 to 255, and smaller to 0
    for mask in masks:
        bin_mask = mask.point(lambda pixel: 0 if pixel < threshold else 255)
        binarized.append(bin_mask)
    
    return binarized

In [8]:
# takes about 2 min
folder = 'DeepGlobe_Road_Extraction/train/'
for img_name in sampled_entries:
    id = img_name.split("_")[0]

    mask_name = id + '_mask.png'

    img_path = folder + img_name
    img_path = os.path.join(os.getcwd(), img_path)

    mask_path = folder  + mask_name
    mask_path = os.path.join(os.getcwd(), mask_path)


    image = Image.open(img_path)
    #resized_image = image.resize((400, 400))
    images = split_in_quarters(image)

    mask = Image.open(mask_path)
    #resized_mask = mask.resize((400, 400))
    resized_grey_mask = mask.convert('L') # convert it to grey scale and dont save colors
    masks = split_in_quarters(resized_grey_mask)
    masks = binarize(masks)
    
    for i in range(0, len(images)):
        
        if check_if_contains_road(masks[i]): # only save if mask has at least 20 pixels with roads
            new_name = 'satimage_' + id + "_" + str(i) + '.png'

            images[i].save('DeepGlobeSampled/images/' + new_name)
            masks[i].save('DeepGlobeSampled/groundtruth/' + new_name)